# Objectif :

Dans ce devoir, vous implémenterez l'algorithme d'apprentissage contrastif auto-supervisé, [SimCLR](https://arxiv.org/abs/2002.05709), en utilisant PyTorch. Vous utiliserez le dataset STL-10 pour ce devoir.

Vous devez compléter la définition de la classe `Net`, la définition de la classe de dataset `SimCLRDataset` et la perte SimCLR dans la classe `Trainer`. Vous devez exécuter la boucle d'entraînement, sauvegarder le meilleur modèle d'entraînement et évaluer en utilisant la tâche de classification `sonde linéaire`. Comme nous n'avons pas assez de ressources GPU et que l'algorithme d'apprentissage contrastif comme SimCLR a généralement besoin d'environ `1000` époques pour s'entraîner (nous n'avons que `70` époques), vous n'obtiendrez peut-être pas les meilleures performances. Ainsi, du côté des performances, tant que vous voyez que la perte diminue (jusqu'à environ 7.4 à `70` époques) et que la précision augmente, c'est bon.

Note :

- **Remplir la définition de la classe Net (5 points).**
- **Remplir la définition de la classe de dataset SimCLRDataset (10 points).**
- **Remplir la perte SimCLR dans la classe Trainer (20 points).**
- **Enregistrer la perte d'entraînement dans les 70 époques, plus elle est basse, mieux c'est (5 points).**
- **Enregistrer la précision de la sonde linéaire, plus elle est haute, mieux c'est (5 points).**
- **Rédiger un rapport incluant :**
  - **Comment vous sélectionnez l'augmentation des données (transformation) dans le pool de transformations.**
  - **Comment vous implémentez la perte SimCLR et expliquez pourquoi votre perte SimCLR est efficace en termes de calcul et équivalente à la fonction de perte dans l'article.**
  - **Inclure la courbe de perte d'entraînement et la précision en aval (15 points). Notez que la logique de journalisation n'est pas fournie, veuillez l'implémenter avant de commencer l'entraînement.**
---
Veuillez NE PAS changer la configuration fournie. Ne changez le code donné que si vous êtes sûr que le changement est nécessaire. Il est recommandé d'**utiliser une session CPU pour déboguer** lorsque le GPU n'est pas nécessaire puisque Colab ne donne que 12 heures d'accès GPU gratuit à la fois. Si vous utilisez toutes les ressources GPU, vous pouvez envisager d'utiliser les ressources GPU de Kaggle. Merci et bonne chance !

# Apprentissage auto-supervisé : SimCLR

Apprentissage auto-supervisé

1.   Concevoir une tâche auxiliaire.
2.   Entraîner le réseau de base sur la tâche auxiliaire.
3.   Évaluer sur la tâche aval : Entraîner un nouveau décodeur basé sur l'encodeur formé.

Plus spécifiquement, comme l'un des algorithmes d'apprentissage auto-supervisé les plus réussis, SimCLR, un algorithme d'apprentissage contrastif, est notre centre d'intérêt aujourd'hui. Ci-dessous, nous allons mettre en œuvre SimCLR comme un exemple d'apprentissage auto-supervisé.

<img src="https://camo.githubusercontent.com/35af3432fbe91c56a934b5ee58931b4848ab35043830c9dd6f08fa41e6eadbe7/68747470733a2f2f312e62702e626c6f6773706f742e636f6d2f2d2d764834504b704539596f2f586f3461324259657276492f414141414141414146704d2f766146447750584f79416f6b4143385868383532447a4f67457332324e68625877434c63424741735948512f73313630302f696d616765342e676966" width="650" height="650">

In [1]:
# Config
# Comme nous utilisons jupyter notebook, nous utilisons easydict pour micic argparse. N'hésitez pas à utiliser d'autres formats de configuration
from easydict import EasyDict
import torch.nn as nn
from tqdm import tqdm
import torch

config = {
    'dataset_name': 'stl10',
    'workers': 1,
    'epochs': 70,
    'batch_size': 3072,
    'lr': 0.0003,
    'weight_decay': 1e-4,
    'seed': 4242,
    'fp16_precision': True,
    'out_dim': 128,
    'temperature': 0.5,
    'n_views': 2,
    'device': "cuda",
}
args = EasyDict(config)

Nous allons utiliser le [dataset STL-10](https://cs.stanford.edu/~acoates/stl10/).

Aperçu

*   10 classes : avion, oiseau, voiture, chat, cerf, chien, cheval, singe, bateau, camion.
*   Les images sont en couleur et mesurent **96x96** pixels.
*   500 images d'entraînement (10 plis pré-définis), 800 images de test par classe.
*   100000 images non étiquetées pour l'apprentissage non supervisé. Ces exemples sont extraits d'une distribution d'images similaire mais plus large. Par exemple, elle contient d'autres types d'animaux (ours, lapins, etc.) et de véhicules (trains, bus, etc.) en plus de ceux du set étiqueté.
*   Les images ont été acquises à partir d'exemples étiquetés sur ImageNet.


## Préparation

Définir un ResNet-18 et une couche MLP supplémentaire comme le modèle d'entraînement dans la tâche auxiliaire.

In [2]:
import torchvision.models as models

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.basemodel = models.resnet18(pretrained=False, num_classes=args.out_dim)
        self.fc_in_features = self.basemodel.fc.in_features
        self.backup_fc = None

        # On ajoute la `projection head g(.)` après le average pooling. 
        # Voici un extrait du papier SimCLR: 
        #  "We use a MLP with one hidden layer to obtain zi = g(hi) = W(2)σ(W(1)hi) where σ is a ReLU nonlinearity. Here hi ∈ Rd is the output after the average pooling layer."
        self.basemodel.fc = nn.Sequential(
            nn.Linear(self.fc_in_features, self.fc_in_features),
            nn.ReLU(),
            self.basemodel.fc
        )

    def forward(self, x):
        return self.basemodel(x)

    def linear_probe(self):
        self.freeze_basemodel_encoder()
        self.backup_fc = self.basemodel.fc  # Sauvegarde de la dernière couche linéaire
        # ToDo: implement la sonde linéaire pour votre tâche en aval. Un prob linéaire est simplement une couche linéaire (pas de MLP, pas de couche d'activation incluse) après le codeur appris.
        self.basemodel.fc = nn.Linear(self.fc_in_features, 10)

    def restore_backbone(self):
        self.basemodel.fc = self.backup_fc
        self.backup_fc = None

    def freeze_basemodel_encoder(self):
        # ne pas geler les poids self.basemodel.fc
        for name, param in self.basemodel.named_parameters():
            if 'fc' not in name:
                param.requires_grad = False

# Étape 1 : Concevoir la tâche auxiliaire.

### Construire le dataset

In [3]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch

#############################################################################
# Calcul des valeurs de la moyenne et de l'écart-type pour la normalisation #
#############################################################################

# # Define a transform to convert images to tensors
# transform = transforms.Compose([
#     transforms.ToTensor()
# ])
# 
# # Load the STL-10 dataset with the defined transform
# dataset = datasets.STL10(root='./datasets', split='unlabeled', download=True, transform=transform)
# 
# # Create a DataLoader
# dataloader = DataLoader(dataset, batch_size=64, shuffle=False)
# 
# def compute_mean_std(loader):
#     channel_sum, channel_squared_sum, num_batches = 0, 0, 0
#     for data, _ in loader:
#         # Rearrange batch to be the shape of [B, C, H, W]
#         data = data.view(data.size(0), data.size(1), -1)
#         # Update sums and squared sums
#         channel_sum += torch.mean(data, dim=[0, 2])
#         channel_squared_sum += torch.mean(data**2, dim=[0, 2])
#         num_batches += 1
#     mean = channel_sum / num_batches
#     std = (channel_squared_sum / num_batches - mean ** 2) ** 0.5
#     return mean, std
# 
# # Compute and print mean and std
# mean, std = compute_mean_std(dataloader)
# print("\n")
# print(f"Mean: {mean.tolist()}")
# print(f"Std: {std.tolist()}")

#############################################################################
# Classes pour le chargement des données et les transformations pour SimCLR #
#############################################################################

class View_sampler(object):
    """This class randomly sample two transforms from the list of transforms for the SimCLR to use. It is used in the SimCLRDataset.get_dataset."""

    def __init__(self, transforms, n_views=2):
        self.transforms = transforms
        self.n_views = n_views

    def __call__(self, x):
        return [self.transforms(x) for i in range(self.n_views)]


class SimCLRDataset:
    def __init__(self, root_folder="./datasets"):
        self.root_folder = root_folder

    @staticmethod
    def transforms_pool(size=96):

        data_transforms = transforms.Compose([
            transforms.RandomResizedCrop(size=(size, size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
            transforms.RandomGrayscale(p=0.2), 
            transforms.RandomApply([transforms.GaussianBlur(11, sigma=(0.1, 2.0))], p=0.5),
            transforms.ToTensor(), 
            transforms.Normalize(mean=[0.44058355689048767, 0.42731037735939026, 0.38579756021499634], std=[0.2686561346054077, 0.2612513303756714, 0.2684949040412903]) # Ces valeurs ont été calculées précédemment
        ])
        
        return data_transforms

    def get_dataset(self):
        dataset_fn = lambda: datasets.STL10(self.root_folder, split='unlabeled', transform=View_sampler(self.transforms_pool(), 2), download=True)
        return dataset_fn()

## Définir le chargeur de données, l'optimiseur et le planificateur

Qu'est-ce qu'un planificateur ?

Un planificateur aide à optimiser la convergence, à éviter les minima locaux et potentiellement à améliorer la performance du modèle sur la tâche donnée. Le taux d'apprentissage est l'un des hyperparamètres les plus importants pour l'entraînement des réseaux neuronaux, et trouver un programme de taux d'apprentissage approprié peut être crucial pour le succès de votre modèle.

En savoir plus ici : https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

In [4]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

model = Net()
dataset = SimCLRDataset()
train_dataset = dataset.get_dataset()

# ToDo, définir un chargeur de données basé sur le train_dataset avec drop_last=True
dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, drop_last=True)

# ToDo, définir un optimiseur avec args.lr comme taux d'apprentissage et args.weight_decay comme weight_decay
optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay) # Utilisé AdamW car il y a une utilisation de weight_decay. On aurait aussi pu utiliser LARS comme dans le papier.

# ToDo, définir un planificateur lr_scheduler CosineAnnealingLR pour l'optimiseur
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(dataloader), eta_min=0, last_epoch=-1)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Files already downloaded and verified


# Définir le formateur

L'Automatic Mixed Precision (AMP) est une technique visant à améliorer la vitesse et l'efficacité de l'entraînement des réseaux neuronaux profonds en utilisant l'entraînement en précision mixte.

**Introduction à l'AMP**

L'AMP permet à l'entraînement des réseaux neuronaux d'utiliser simultanément l'arithmétique à simple précision (FP32) et à demi-précision (FP16). L'idée principale derrière l'AMP est de réaliser certaines opérations en FP16 pour exploiter l'arithmétique plus rapide et la réduction de l'utilisation de la mémoire de calcul à précision inférieure, tout en maintenant les parties critiques du calcul en FP32 pour assurer la précision et la stabilité du modèle.

**Pourquoi Nous Ne Pouvons Pas Toujours Utiliser le FP16**



*   **Stabilité Numérique** : Le FP16 a une plage dynamique plus petite et une précision inférieure par rapport au FP32. Cette limitation peut conduire à une instabilité numérique, telle que des sous-débordements et des surdébordements, en particulier pendant des opérations qui impliquent de petites valeurs de gradient ou nécessitent une haute précision numérique. Cela peut affecter négativement la convergence et la précision du modèle entraîné.
*   **Exigences Sélectives de Précision** : Certaines opérations et couches au sein des réseaux neuronaux sont plus sensibles à la précision que d'autres. Par exemple, les mises à jour de poids dans les optimiseurs peuvent nécessiter le FP32 pour maintenir la précision dans le temps. Les stratégies AMP impliquent donc d'appliquer sélectivement le FP16 à des parties du calcul où cela peut être bénéfique sans compromettre l'ensemble du processus d'entraînement.

Ci-dessous, nous présentons comment inclure la logique AMP dans la procédure d'entraînement standard de torch.

Avant d'inclure AMP :
```python
for batch in data_loader:
    # Forward pass
    inputs, targets = batch
    outputs = model(inputs)
    loss = loss_fn(outputs, targets)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
```

Après avoir inclus AMP :
```python
from torch.cuda.amp import GradScaler, autocast

scaler = GradScaler()

for batch in data_loader:
    inputs, targets = batch[0].cuda(), batch[1].cuda()

    # Forward pass
    with autocast():
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

    # Backward pass and optimize
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
```


Read more here: https://pytorch.org/docs/stable/amp.html

## Implémenter la fonction de perte dans l'entraîneur
L'algorithme de la fonction de perte SimCLR est le suivant :

$$
\begin{aligned}
&\text{for all } i \in \{1, \ldots, 2N\} \text{ and } j \in \{1, \ldots, 2N\} \text{ do} \\
&\quad s_{i,j} = \frac{z_i^\top z_j}{\|z_i\|\|z_j\|} \quad \text{pairwise similarity} \\
&\text{end for} \\
&\text{define } \ell(i,j) \text{ as } \ell(i,j) = -\log \left( \frac{\exp(s_{i,j} / \tau)}{\sum_{k=1}^{2N} \mathbb{1}_{[k \neq j]} \exp(s_{i,k} / \tau)} \right) \\
&\mathcal{L} = \frac{1}{2N} \sum_{k=1}^{N} \left[\ell(2k-1, 2k) + \ell(2k, 2k-1)\right] \\
&\text{update networks } f \text{ and } g \text{ to minimize } \mathcal{L}
\end{aligned}
$$


Veuillez remplir les blancs de la fonction de perte ci-dessous. Conseil : mettez en place un masque pour éviter d'inclure la similarité entre soi et soi, ainsi que les paires positives et les paires négatives.

In [5]:
import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

class Trainer():
    def __init__(self, *args, **kwargs):
        self.args = kwargs['args']
        self.model = kwargs['model'].to(self.args.device)
        self.optimizer = kwargs['optimizer']
        self.scheduler = kwargs['scheduler']
        self.criterion = torch.nn.CrossEntropyLoss().to(self.args.device)
        self.losses = []

    def loss(self, features):
        # Les caractéristiques d'entrée sont un tenseur de torche avec une forme de (2*batch_size, out_dim)
        # Les paires positives sont (features[i] et features[i+batch_size]) pour tout i
        
        # Les étiquettes sont simplement les indices de la classe pour chaque image
        batch_indices = torch.arange(self.args.batch_size).repeat(self.args.n_views)
        labels = torch.eq(batch_indices.unsqueeze(0), batch_indices.unsqueeze(1)).float().to(self.args.device)
        
        # Normaliser les vecteurs pour avoir une norme de 1
        normalized_features = F.normalize(features, dim=1)

        # Calculer la matrice de similarité (cosine similarity)
        similarity = torch.mm(normalized_features, normalized_features.T)

        # Enlever les éléments diagonaux, car ils correspondent à la similarité de l'image avec elle-même
        diagonal_mask = torch.eye(labels.size(0), dtype=torch.bool).to(self.args.device)
        labels_filtered = labels.masked_select(~diagonal_mask).view(labels.size(0), -1)
        similarity_filtered = similarity.masked_select(~diagonal_mask).view(similarity.size(0), -1)

        # Sélectionner uniquement les positifs et négatifs sans besoin de reformater comme précédemment
        positive_pairs = similarity_filtered[labels_filtered.bool()].view(labels.size(0), -1)
        negative_pairs = similarity_filtered[~labels_filtered.bool()].view(labels.size(0), -1)

        # Préparer les logits pour le calcul de perte
        logits = torch.cat([positive_pairs, negative_pairs], dim=1) / self.args.temperature
        zero_labels = torch.zeros(logits.size(0), dtype=torch.long).to(self.args.device)
        
        return self.criterion(logits, zero_labels)

    def train(self, dataloader):
        # Implement GradScaler if AMP
        best_loss = 1e4
        scaler = GradScaler(enabled=self.args.fp16_precision)
        for epoch in range(self.args.epochs):
            for images, _ in tqdm(dataloader):
                images = torch.cat(images, dim=0)
                images = images.to(self.args.device)

                with autocast(enabled=self.args.fp16_precision):
                    features = self.model(images)
                    loss = self.loss(features)
                    self.losses.append(loss.item())
                    
                self.optimizer.zero_grad()

                scaler.scale(loss).backward()
                scaler.step(self.optimizer)
                scaler.update()

                self.scheduler.step()

            # Warmup for the first 10 epochs (échauffement pour les 10 premières époques)
            if epoch >= 10:
                self.scheduler.step()
            if epoch % 10 == 0 and epoch != 0:
                self.save_model(self.model, f"./HW3_Representation/simclr_models/model_{epoch}.pth")
            if loss < best_loss:
                best_loss = loss
                self.save_model(self.model, f"./HW3_Representation/simclr_models/best_model.pth")
            print(f"Epoch {epoch}, Loss {loss.item()}")
        
        return self.model, self.losses

    def save_model(self, model, path):
        torch.save(model.state_dict(), path)

# Étape 2 : Entraînez le réseau de base sur la tâche auxiliaire pendant 70 époques et sauvegardez le meilleur modèle que vous avez pour l'évaluation.

Vérifiez si la perte d'entraînement diminue avec le temps et essayez de capturer d'autres bogues possibles à l'aide d'outils de journalisation.

In [6]:
import pickle
import os

torch.autograd.set_detect_anomaly(True)

trainer = Trainer(args=args, model=model, optimizer=optimizer, scheduler=lr_scheduler)
model, losses = trainer.train(dataloader)

# Save the losses list to a file
with open('./HW3_Representation/simclr_models/losses.pkl', 'wb') as f:
    pickle.dump(losses, f)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [05:11<00:00,  9.75s/it]


Epoch 0, Loss 8.21781063079834


100%|██████████| 32/32 [05:11<00:00,  9.73s/it]


Epoch 1, Loss 8.148459434509277


...

Epoch 67, Loss 7.409276485443115


100%|██████████| 32/32 [05:13<00:00,  9.80s/it]


Epoch 68, Loss 7.406470775604248


100%|██████████| 32/32 [05:10<00:00,  9.71s/it]

Epoch 69, Loss 7.4189019203186035


# Étape 3 : Évaluation de la tâche en aval : Former un nouveau décodeur MLP basé sur le codeur formé.

Ce processus de fine-tuning devrait être bien plus rapide que le précédent. L'exactitude Top-1 attendue devrait être d'environ 57% et l'exactitude Top-5 devrait être autour de 97%. Obtenir ces résultats est normal car la sonde linéaire est juste une couche de projection généralement reconnue comme n'ayant pas de capacité de représentation. Pour atteindre la performance mentionnée dans l'article, nous avons besoin d'un plus grand dataset, de GPU plus puissants et de plus de temps (environ 1000 époques durant la phase de pré-entraînement).

In [7]:
class linear_prob_Trainer:
    def __init__(self, *args, **kwargs):
        self.args = kwargs["args"]
        self.model = kwargs["model"].to(self.args.device)
        self.optimizer = kwargs["optimizer"]
        self.criterion = torch.nn.CrossEntropyLoss().to(self.args.device)
        self.train_dataset = datasets.STL10(
            "./data", split="train", download=True, transform=transforms.ToTensor()
        )

        self.train_loader = torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.args.batch_size,
            num_workers=1,
            drop_last=False,
        )

        self.test_dataset = datasets.STL10(
            "./data", split="test", download=True, transform=transforms.ToTensor()
        )

        self.test_loader = torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.args.batch_size,
            num_workers=1,
            drop_last=False,
        )

    def accuracy(self, output, target, topk=(1,)):
        with torch.no_grad():
            maxk = max(topk)
            batch_size = target.size(0)

            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))

            res = []
            for k in topk:
                correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
                res.append(correct_k.mul_(100.0 / batch_size))
            return res

    def train(self, dataloader):
        top1_accs = []
        top5_accs = []
        for epoch in range(100):
            top1_train_accuracy = 0
            top5_train_accuracy = 0
            for images, labels in tqdm(dataloader):
                images, labels = images.to(self.args.device), labels.to(
                    self.args.device
                )
                logits = self.model(images)
                loss = self.criterion(logits, labels)
                
                top1 = self.accuracy(logits, labels, topk=(1,))
                top1_train_accuracy += top1[0]
                
                top5 = self.accuracy(logits, labels, topk=(5,))
                top5_train_accuracy += top5[0]

                top1_accs.append(top1[0])
                top5_accs.append(top5[0])

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                print(
                    f"Epoch: {epoch}, Loss: {loss.item()}",
                    "Top1 Train Accuracy: ",
                    top1_train_accuracy.item() / len(dataloader),
                    "Top5 Train Accuracy: ",
                    top5_train_accuracy.item() / len(dataloader),
                )
        return self.model, top1_accs, top5_accs

    def test(self, dataloader):
        
        with torch.no_grad():
            model.eval()
            top1_test_accuracy = 0
            top5_test_accuracy = 0
            for images, labels in tqdm(dataloader):
                images, labels = images.to(self.args.device), labels.to(
                    self.args.device
                )
                logits = self.model(images)
                top1 = self.accuracy(logits, labels, topk=(1,))
                top1_test_accuracy += top1[0]
                top5 = self.accuracy(logits, labels, topk=(5,))
                top5_test_accuracy += top5[0]
            top1_acc = top1_test_accuracy.item() / len(dataloader)
            top5_acc = top5_test_accuracy.item() / len(dataloader)
            print("Top1 Test Accuracy: ", top1_acc)
            print("Top5 Test Accuracy: ", top5_acc)
            return top1_acc, top5_acc

model = Net()
model.load_state_dict(torch.load("./HW3_Representation/simclr_models/best_model.pth"))

model.linear_probe()
linear_probe_optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
linear_prob_trainer = linear_prob_Trainer(args=args, model=model, optimizer=linear_probe_optimizer)
model, top1_train, top5_train = linear_prob_trainer.train(linear_prob_trainer.train_loader)
top1_test, top5_test = linear_prob_trainer.test(linear_prob_trainer.test_loader)

100%|██████████| 2640397119/2640397119 [01:49<00:00, 24105813.32it/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


 50%|█████     | 1/2 [00:01<00:01,  1.46s/it]

Epoch: 0, Loss: 2.550807237625122 Top1 Train Accuracy:  4.8177080154418945 Top5 Train Accuracy:  24.348957061767578


100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


...

Epoch: 99, Loss: 1.121546983718872 Top1 Train Accuracy:  29.638669967651367 Top5 Train Accuracy:  48.81184768676758


100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


Epoch: 99, Loss: 1.0860403776168823 Top1 Train Accuracy:  59.59198760986328 Top5 Train Accuracy:  97.56703186035156


100%|██████████| 3/3 [00:02<00:00,  1.12it/s]

Top1 Test Accuracy:  57.094482421875
Top5 Test Accuracy:  96.75302124023438
